###  数据重复会导致数据的方差变小，数据分布发生较大变化。缺失会导致样本信息减少，不仅增加了数据分析的难度，而且会导致数据分析的结果产生偏差。异常值则会产生“伪回归”。因此需要对数据进行检测，查询是否有重复值、缺失值和异常值，并且要对这些数据进行适当的处理。

##  1.检测与处理重复值

### 1.1记录重复值

####  菜品订单详情表中的dishes_name特征存放了每个订单的菜品。要找出所有已点菜品，最简单的方法就是利用去重操作实现。

#### 方法一：利用list 列表去重。

####  代码5-9 利用list 列表去重

In [2]:
import pandas as pd
detail = pd.read_csv('./data/detail.csv',
    index_col=0,encoding = 'gbk')

##方法一
##定义去重函数
def delRep(list1):
    list2=[]
    for i in list1:
        if i not in list2:
            list2.append(i)
    return list2 
## 去重
dishes=list(detail['dishes_name']) ##将dishes_name从数据框中提取出来
print('去重前菜品总数为：',len(dishes)) 
dish = delRep(dishes) ##使用自定义的去重函数去重
print('方法一去重后菜品总数为：',len(dish))


去重前菜品总数为： 10037
方法一去重后菜品总数为： 145


#### 方法二：利用set集合去重。

####  代码5-10 利用set集合去重

In [3]:
print('去重前菜品总数为：',len(dishes)) 
dish_set = set(dishes) ##利用set的特性去重
print('方法二去重后菜品总数为：',len(dish_set))


去重前菜品总数为： 10037
方法二去重后菜品总数为： 145


####  比较上述两种方法可以发现，代码5-9中的方法明显代码冗长，会拖慢数据分析的整体进度。                                                                                     代码5-10 使用了集合的元素的唯一性，看似代码简洁多了。但这种方法的最大问题是会导致数据的排列发生变化。

### pandas提供了一个名为drop_duplicates的去重方法。该方法只对DataFrame或者Series类型有效。这种方法不会改变数据原始排列，并且代码简洁和运行稳定。                                                                                                                                               
### pandas.DataFrame(Series).drop_drop_duplicates(self,subset=None,keep='first',inplace=False)

###  使用该方法去重时，当且仅当subset参数中的特征重复的时候才会执行去重操作，去重时可以选择保留哪一个，甚至可以不保留。

####  代码5-11利用drop_duplicates方法对菜品名称去重（菜品订单详情表）

In [4]:
##对dishes_name去重
dishes_name = detail['dishes_name'].drop_duplicates()
print('drop_duplicates方法去重之后菜品总数为：',len(dishes_name))

drop_duplicates方法去重之后菜品总数为： 145


####  代码5-12利用drop_duplicates方法对多列去重（菜品订单详情表）

In [5]:
print('去重之前订单详情表的形状为：', detail.shape)
shapeDet = detail.drop_duplicates(subset = ['order_id',
    'emp_id']).shape
print('依照订单编号，会员编号去重之后订单详情表大小为:', shapeDet)


去重之前订单详情表的形状为： (10037, 18)
依照订单编号，会员编号去重之后订单详情表大小为: (942, 18)



### 1.2 特征重复

#### 结合相关的数学和统计学知识，去除连续型特征重复可以利用特征间的相似度将两个相似度为1的特征去除一个。在pandas中相似度的计算方法为corr，使用该方法计算相似度时，默认为“pearson”法 ，可以通过“method”参数调节，目前还支持“spearman”法和“kendall”法。

####  代码5-13 使用kendall法求出菜品订单详情表(detail.csv)数据中counts和amounts列的相似度矩阵

In [6]:
## 求取销量和售价的相似度
corrDet = detail[['counts','amounts']].corr(method='kendall')
print('销量和售价的kendall相似度为：\n',corrDet)


销量和售价的kendall相似度为：
            counts   amounts
counts   1.000000 -0.229968
amounts -0.229968  1.000000


* 但是通过相似度矩阵去重存在一个弊端，该方法只能对数值型重复特征去重，类别型特征之间无法通过计算相似系数来衡量相似度。

####  对订单详情表(detail.csv)中'dishes_name','counts', 'amounts' 这3个特征进行  “pearson”法 相似度矩阵的求解，但是最终只存在'counts'和'amounts'特征的2*2的相似度矩阵

####  代码5-14 'dishes_name','counts', 'amounts' 这3个特征进行  “pearson”法 相似度矩阵的求解

In [7]:
corrDet1 = detail[['dishes_name','counts',
    'amounts']].corr(method='pearson')
print('菜品名称，销量和售价的pearson相似度为：\n',corrDet1)


菜品名称，销量和售价的pearson相似度为：
            counts   amounts
counts   1.000000 -0.159264
amounts -0.159264  1.000000


#### 除了使用相似度矩阵进行特征去重之外，可以通过DataFrame.equals的方法进行特征去重。

####  代码5-15 使用过DataFrame.equals的方法进行特征去重

In [8]:
##定义求取特征是否完全相同的矩阵的函数
def FeatureEquals(df):
    dfEquals=pd.DataFrame([],columns=df.columns,index=df.columns)
    for i in df.columns:
       for j in df.columns:
           dfEquals.loc[i,j]=df.loc[:,i].equals(df.loc[:,j])
    return dfEquals
## 应用上述函数
detEquals=FeatureEquals(detail)
print('detail的特征相等矩阵的前5行5列为：\n',detEquals.iloc[:5,:5])


detail的特征相等矩阵的前5行5列为：
                    order_id  dishes_id  logicprn_name  parent_class_name  \
order_id               True      False          False              False   
dishes_id             False       True          False              False   
logicprn_name         False      False           True               True   
parent_class_name     False      False           True               True   
dishes_name           False      False          False              False   

                   dishes_name  
order_id                 False  
dishes_id                False  
logicprn_name            False  
parent_class_name        False  
dishes_name               True  


####  再通过遍历的方式筛出完全重复的特征。

####  代码5-16 通过遍历的方式进行数据筛选

In [9]:
##遍历所有数据
lenDet = detEquals.shape[0]
dupCol = []
for k in range(lenDet):
    for l in range(k+1,lenDet):
        if detEquals.iloc[k,l] & (detEquals.columns[l] not in dupCol):
            dupCol.append(detEquals.columns[l])
##进行去重操作
print('需要删除的列为：',dupCol)
detail.drop(dupCol,axis=1,inplace=True)
print('删除多余列后detail的特征数目为：',detail.shape[1])

需要删除的列为： ['parent_class_name', 'cost', 'discount_amt', 'discount_reason', 'kick_back', 'add_info', 'bar_code', 'add_inprice']
删除多余列后detail的特征数目为： 10


##  2.检测与处理缺失值

#### 数据中的某个或某些特征的值是不完整的，这些值称为缺失值。
#### pandas提供了识别缺失值的方法isnull以及识别非缺失值的方法notnull，这两种方法在使用时返回的都是布尔值True和False。
#### 结合sum函数和isnull、notnull函数，可以检测数据中缺失值的分布以及数据中一共含有多少缺失值。

#### 代码5-17 isnull和notnull用法

In [10]:
print('detail每个特征缺失的数目为：\n',detail.isnull().sum())
print('detail每个特征非缺失的数目为：\n',detail.notnull().sum())

detail每个特征缺失的数目为：
 order_id                0
dishes_id               0
logicprn_name       10037
dishes_name             0
itemis_add              0
counts                  0
amounts                 0
place_order_time        0
picture_file            0
emp_id                  0
dtype: int64
detail每个特征非缺失的数目为：
 order_id            10037
dishes_id           10037
logicprn_name           0
dishes_name         10037
itemis_add          10037
counts              10037
amounts             10037
place_order_time    10037
picture_file        10037
emp_id              10037
dtype: int64


#### isnull和notnull之间结果正好相反，因此使用其中任意一个都可以判断出数据中缺失值的位置。


###   2.1 删除法

#### 删除法分为删除观测记录和删除特征两种，它属于利用减少样本量来换取信息完整度的一种方法，是一种最简单的缺失值处理方法。
#### pandas中提供了简便的删除缺失值的方法dropna，该方法既可以删除观测记录，亦可以删除特征。
#### pandas.DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)

#### 代码5-18 对菜品订单详情表利用dropna方法删除缺失值

In [11]:
print('去除缺失的列前detail的形状为：', detail.shape)
print('去除缺失的列后detail的形状为：',
    detail.dropna(axis = 1,how ='any').shape)

去除缺失的列前detail的形状为： (10037, 10)
去除缺失的列后detail的形状为： (10037, 9)


#### 当how参数取值为any时，删除了一个特征，说明这个特征存在缺失值。若how参数不取any这个默认值，而是取all，则表示整个特征全部为缺失值时才会执行删除操作。

###   2.2替换法

#### 替换法是指用一个特定的值替换缺失值。
#### 特征可分为数值型和类别型，两者出现缺失值时的处理方法也是不同的。
#### 缺失值所在特征为数值型时，通常利用其均值、中位数和众数等描述其集中趋势的统计量来代替缺失值。
#### 缺失值所在特征为类别型时，则选择使用众数来替换缺失值。

#### 代码5-19使用fillna方法替换缺失值

In [12]:
detail = detail.fillna(-99)
print('detail每个特征缺失的数目为：\n',detail.isnull().sum())

detail每个特征缺失的数目为：
 order_id            0
dishes_id           0
logicprn_name       0
dishes_name         0
itemis_add          0
counts              0
amounts             0
place_order_time    0
picture_file        0
emp_id              0
dtype: int64


###   2.3 插值法


#### 代码5-20 SCIPy interpolate模块插值

In [13]:
## 线性插值
import numpy as np
from scipy.interpolate import interp1d
x=np.array([1,2,3,4,5,8,9,10]) ##创建自变量x
y1=np.array([2,8,18,32,50,128,162,200]) ##创建因变量y1
y2=np.array([3,5,7,9,11,17,19,21]) ##创建因变量y2
LinearInsValue1 = interp1d(x,y1,kind='linear') ##线性插值拟合x,y1
LinearInsValue2 = interp1d(x,y2,kind='linear') ##线性插值拟合x,y2
print('当x为6、7时，使用线性插值y1为：',LinearInsValue1([6,7]))
print('当x为6、7时，使用线性插值y2为：',LinearInsValue2([6,7]))


当x为6、7时，使用线性插值y1为： [ 76. 102.]
当x为6、7时，使用线性插值y2为： [13. 15.]


In [14]:
## 拉格朗日插值
from scipy.interpolate import lagrange
LargeInsValue1 = lagrange(x,y1) ##拉格朗日插值拟合x,y1
LargeInsValue2 = lagrange(x,y2) ##拉格朗日插值拟合x,y2
print('当x为6,7时，使用拉格朗日插值y1为：',LargeInsValue1([6,7]))
print('当x为6,7时，使用拉格朗日插值y2为：',LargeInsValue2([6,7]))


当x为6,7时，使用拉格朗日插值y1为： [72. 98.]
当x为6,7时，使用拉格朗日插值y2为： [13. 15.]


In [15]:
##样条插值
from scipy.interpolate import spline
##样条插值拟合x,y1
SplineInsValue1 = spline(x,y1,xnew=np.array([6,7]))
##样条插值拟合x,y2
SplineInsValue2 = spline(x,y2,xnew=np.array([6,7]))
print('当x为6,7时，使用样条插值y1为：',SplineInsValue1)
print('当x为6,7时，使用样条插值y2为：',SplineInsValue2)


当x为6,7时，使用样条插值y1为： [72. 98.]
当x为6,7时，使用样条插值y2为： [13. 15.]


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `spline` is deprecated!
spline is deprecated in scipy 0.19.0, use Bspline class instead.
  after removing the cwd from sys.path.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: `spline` is deprecated!
spline is deprecated in scipy 0.19.0, use Bspline class instead.
  


##  3.检测与处理异常值

###   3.1  3σ原则

####  代码 5-21 使用3σ原则识别异常值

In [16]:

## 定义拉依达准则识别异常值函数
def outRange(Ser1):
    boolInd = (Ser1.mean()-3*Ser1.std()>Ser1) | \
    (Ser1.mean()+3*Ser1.var()< Ser1)
    index = np.arange(Ser1.shape[0])[boolInd]
    outrange = Ser1.iloc[index]
    return outrange
outlier = outRange(detail['counts'])
print('使用拉依达准则判定异常值个数为:',outlier.shape[0])
print('异常值的最大值为：',outlier.max())
print('异常值的最小值为：',outlier.min())


使用拉依达准则判定异常值个数为: 209
异常值的最大值为： 10
异常值的最小值为： 3


####  3σ原则具有一定的局限性，即此原则只对正态分布或近似正态分布的数据有效，而对其分布类型的数据无效

###   3.2.箱线图分析

####  代码 5-22菜品售价根据箱线图识别异常值

In [17]:
# 代码 5-22
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8)) 
p = plt.boxplot(detail['counts'].values,notch=True)   ##画出箱线图
outlier1 = p['fliers'][0].get_ydata()   ##fliers为异常值的标签
plt.savefig('../tmp/菜品异常数据识别.png')
plt.show()
print('销售量数据异常值个数为：',len(outlier1))
print('销售量数据异常值的最大值为：',max(outlier1))
print('销售量数据异常值的最小值为：',min(outlier1))

<Figure size 1000x800 with 1 Axes>

销售量数据异常值个数为： 516
销售量数据异常值的最大值为： 10
销售量数据异常值的最小值为： 2


####  可以从结果中看出，菜品订单表中的菜品售价存在着为数不少的异常值。